# **1 - Pre-processing:**

## **Install and import of necessary libraries:**

In [ ]:
# installing necessary libraries that are not already installed by default on Google Colab:

# library to read .ods files:
!pip install odfpy;
print('')
print('===============')
print('')

# library to read datasets directly from Google Big Query:
!pip install basedosdados;
#!pip install google-resumable-media
#!pip install --upgrade googleapis-common-protos
print('')
print('===============')
print('')

# library to make more advanced data visualizations:
!pip install plotly_express;

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 717 kB 33.2 MB/s 
  Created wheel for odfpy: filename=odfpy-1.4.1-py2.py3-none-any.whl size=160692 sha256=b976fa28843d5d685dc9d00c7b4b4a33c122a9ca45d69e6d445f77a678a6eb39
  Stored in directory: /root/.cache/pip/wheels/e2/f4/5d/a68c656235d33455a1d0f78e877acddfa006907a6d52d7e6ee
Successfully built odfpy


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49 kB 4.6 MB/s 
     |████████████████████████████████| 1.1 MB 26.0 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 120 kB 54.2 MB/s 
     |████████████████████████████████| 108 kB 61.8 MB/s 
     |████████████████████████████████| 203 kB 53.4 MB/s 
     |████████████████████████████████| 270



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# importing libraries:

# basic libraries:
import numpy as np
import pandas as pd

# data import libraries:
import basedosdados as bd
from google.colab import drive
import glob

# data visualization libraries
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly_express as px # bubble chart

## **Declaration of functions:**

In [ ]:
def gera_df_pop(drive_path):

    df_municipio = pd.read_excel(f'{drive_path}/datasets/planilhas_municipios/RELATORIO_DTB_BRASIL_MUNICIPIO.ods')

    df_populacao = pd.read_csv(f'{drive_path}/datasets/planilhas_populacao/populacao.csv')

    df = pd.merge(left=df_populacao, right=df_municipio, how='inner', left_on='id_municipio', right_on='Código Município Completo')

    cols = [
        'ano', 'id_municipio', 'Nome_Município', 'Nome_UF',
        'Nome Região Geográfica Intermediária', 'Nome Região Geográfica Imediata', 'Nome_Mesorregião', 'Nome_Microrregião',
        'sexo', 'grupo_idade', 'populacao'
    ]
    df = df[cols]

    df.columns = [
        'ano', 'id_municipio', 'nome_municipio', 'uf',
        'regiao_intermediaria', 'regiao_imediata', 'mesorregiao', 'microrregiao',
        'genero', 'grupo_idade', 'populacao'
    ]

    # pegando apenas os 6 primeiros digitos do id, pra poder cruzar com os outros datasets:
    df['id_municipio'] = df['id_municipio'].apply(lambda x: str(x)[0:6])

    return df

def consolida_populacao_por_municipio(drive_path):

    df = gera_df_pop(drive_path)

    group_cols = ['ano', 'id_municipio', 'uf', 'regiao_intermediaria', 'regiao_imediata', 'mesorregiao', 'microrregiao', 'nome_municipio']

    df_group = df.groupby(by=group_cols, as_index=False).agg({
        'populacao': ['count', 'sum', 'mean', 'median', 'std']
    })

    df_group.columns = df_group.columns.droplevel(0)
    group_cols.extend(['qtde', 'pop_total', 'pop_media', 'pop_mediana', 'pop_std'])
    df_group.columns = group_cols

    return df_group


###############################

def gera_df_beneficiarios(file_path):

    df_beneficiarios = pd.read_csv(file_path, sep=';', encoding='ISO-8859-1')
    df_beneficiarios['CD_MUNICIPIO'] = df_beneficiarios['CD_MUNICIPIO'].astype(str)
    df_beneficiarios = df_beneficiarios.query("DT_INCLUSAO < '2022-01'")

    return df_beneficiarios


def consolida_beneficiarios_por_municipio(file_path):

    df = gera_df_beneficiarios(file_path)

    df_group = df.groupby(by=['CD_MUNICIPIO'], as_index=False).agg({
        'CD_OPERADORA': ['count', 'nunique'],
        'DT_INCLUSAO': ['nunique', 'min', 'max']
    })

    df_group.columns = df_group.columns.droplevel(0)
    df_group.columns = ['id_municipio', 'qtde_beneficiarios', 'qtde_operadoras', 'qtde_datas', 'min_data', 'max_data']
    df_group['file_path'] = file_path

    return df_group

def junta_beneficiarios_todos_estados(drive_path):

    all_files = glob.glob(f'{drive_path}/SIB/sib_202203_[PGEMB]*.csv')

    df = pd.DataFrame()
    for file_path in all_files:
        df_temp = consolida_beneficiarios_por_municipio(file_path)

        df = pd.concat([df, df_temp], axis=0, ignore_index=True)

    #df = pd.concat((consolida_beneficiarios_por_municipio(file_path) for file_path in all_files))

    return df

###############################

def gera_df_estabelecimentos(drive_path):

    df_estabelecimentos = pd.read_csv(f'{drive_path}/CNES/bq-results-20220702-123105-1656765097865.csv')
    df_estabelecimentos['id_municipio_6'] = df_estabelecimentos['id_municipio_6'].astype(str)

    return df_estabelecimentos


def consolida_estabelecimentos_por_municipio(drive_path):

    df = gera_df_estabelecimentos(drive_path)

    df_group = df.groupby(by=['id_municipio_6'], as_index=False).agg({
        'id_cnes': ['count'],
        'tipo_unidade': ['nunique']
    })
    df_group.columns = df_group.columns.droplevel(0)
    df_group.columns = ['id_municipio', 'qtde_estabelecimentos', 'qtde_tipos']

    return df_group


###############################


def gera_df_pib(drive_path):

    df_pib = pd.read_csv(f'{drive_path}/PIB_Municipios_2019.csv', sep=';')
    df_pib['PIB_PRC_CORRENTE'] = df_pib['PIB_PRC_CORRENTE'].apply(lambda x: x.replace('.', '').replace(',', '.'))
    df_pib['PIB_PERCAPITA_PRC_CORRENTE'] = df_pib['PIB_PERCAPITA_PRC_CORRENTE'].apply(lambda x: x.replace('.', '').replace(',', '.'))

    df_pib['PIB_PRC_CORRENTE'] = df_pib['PIB_PRC_CORRENTE'].astype(float)
    df_pib['PIB_PERCAPITA_PRC_CORRENTE'] = df_pib['PIB_PERCAPITA_PRC_CORRENTE'].astype(float)

    df_pib['COD_MUNICIPIO'] = df_pib['COD_MUNICIPIO'].astype(str)
    df_pib['COD_MUNICIPIO_6'] = df_pib['COD_MUNICIPIO_6'].astype(str)

    df_pib.drop(['ANO', 'COD_MUNICIPIO'], axis=1, inplace=True)
    df_pib.columns = ['id_municipio', 'valor_pib', 'pib_per_capita']

    return df_pib


###############################

def junta_datasets(df_pop, df_beneficiarios, df_estabelecimentos, df_pib, ano=2021):

    df_pop = df_pop.query(f"ano == {ano}")

    df_merge = pd.merge(left=df_pop, right=df_beneficiarios, how='left', on='id_municipio')

    df_merge = pd.merge(left=df_merge, right=df_estabelecimentos, how='left', on='id_municipio')

    df_final = pd.merge(left=df_merge, right=df_pib, how='left', on='id_municipio')


    df_final['proporcao_beneficiarios'] = df_final['qtde_beneficiarios'] / df_final['pop_total'] * 100
    df_final['cidadao_por_hospital'] = df_final['pop_total'] / df_final['qtde_estabelecimentos']
    df_final['beneficiario_por_hospital'] = df_final['qtde_beneficiarios'] / df_final['qtde_estabelecimentos']

    cols = [
        'id_municipio', 'uf', 'nome_municipio', 'regiao_intermediaria', 'regiao_imediata', 'mesorregiao', 'microrregiao',
        'pop_total', 'qtde_beneficiarios', 'qtde_operadoras', 'proporcao_beneficiarios',
        'qtde_estabelecimentos', 'cidadao_por_hospital', 'beneficiario_por_hospital',
        'valor_pib', 'pib_per_capita'
    ]
    df_final = df_final[cols]

    return df_final


###############################

def monta_abt(drive_path):

    df_pop = consolida_populacao_por_municipio(drive_path)
    df_beneficiarios = junta_beneficiarios_todos_estados(drive_path)
    df_estabelecimentos = consolida_estabelecimentos_por_municipio(drive_path)
    df_pib = gera_df_pib(drive_path)

    df = junta_datasets(df_pop, df_beneficiarios, df_estabelecimentos, df_pib, ano=2021)

    return df


def pivot_pop_por_genero(df):

    df_pivot_genero = pd.pivot_table(df, values='populacao', index=['id_municipio'],
                        columns=['genero'], aggfunc=np.sum)

    df_pivot_genero.columns = [col for col in df_pivot_genero.columns]
    df_pivot_genero.reset_index(inplace=True)
    df_pivot_genero['id_municipio'] = df_pivot_genero['id_municipio'].astype(str)

    return df_pivot_genero


def pivot_pop_por_idade(df):

    df_pivot_idade = pd.pivot_table(df, values='populacao', index=['id_municipio'],
                        columns=['grupo_idade'], aggfunc=np.sum)

    df_pivot_idade.columns = [col for col in df_pivot_idade.columns]
    df_pivot_idade.reset_index(inplace=True)
    df_pivot_idade['id_municipio'] = df_pivot_idade['id_municipio'].astype(str)

    return df_pivot_idade



def gera_df_pivot_pop(drive_path):

    df_pop = gera_df_pop(drive_path)
    df_pop_2021 = df_pop.query("ano == 2021")
    df_pivot_genero = pivot_pop_por_genero(df_pop_2021)
    df_pivot_idade = pivot_pop_por_idade(df_pop_2021)

    df_pivot_pop = pd.merge(left=df_pivot_genero, right=df_pivot_idade, how='inner', on='id_municipio')

    return df_pivot_pop


def pivot_beneficiario_por_genero(df):

    df_pivot_genero = pd.pivot_table(df, values='LG_BENEFICIARIO_ATIVO', index=['CD_MUNICIPIO'],
                        columns=['TP_SEXO'], aggfunc=np.sum) #, margins=True)

    df_pivot_genero.columns = [col for col in df_pivot_genero.columns]
    df_pivot_genero.reset_index(inplace=True)
    df_pivot_genero.columns = ['id_municipio', 'qtde_beneficiarios_feminino', 'qtde_beneficiarios_masculino']
    df_pivot_genero['id_municipio'] = df_pivot_genero['id_municipio'].astype(str)

    return df_pivot_genero


def pivot_estabelecimento_por_tipo(drive_path):

    df_estabelecimentos = gera_df_estabelecimentos(drive_path)
    df_estabelecimentos = df_estabelecimentos[[
        'id_municipio_6', 'vinculo_sus', 'tipo_unidade', 'id_cnes', 'cnpj_mantenedora',
        'atendimento_internacao_particular', 'atendimento_amb_particular'
    ]]
    df_estabelecimentos.rename({'id_municipio_6': 'id_municipio'}, inplace=True, axis=1)

    df_tipo_unidade = pd.read_csv(f'{drive_path}/bq-results-20220702-130407-1656767071622.csv')
    df_tipo_unidade = df_tipo_unidade[['chave', 'valor']]
    df_tipo_unidade.columns = ['tipo_unidade', 'nome_tipo_unidade']

    df_estabelecimentos = pd.merge(left=df_estabelecimentos, right=df_tipo_unidade, how='left', on='tipo_unidade')

    df_pivot_tipo = pd.pivot_table(df_estabelecimentos, values='id_cnes', index=['id_municipio'],
                        columns=['nome_tipo_unidade'], aggfunc='count', fill_value=0)

    df_pivot_tipo.columns = [col for col in df_pivot_tipo.columns]
    df_pivot_tipo.reset_index(inplace=True)
    df_pivot_tipo['id_municipio'] = df_pivot_tipo['id_municipio'].astype(str)

    return df_pivot_tipo


def pivot_profissionais_por_tipo(drive_path):

    df_profissionais = pd.read_csv(f'{drive_path}/profissional_cnes.csv')

    df_codigo = pd.read_csv(f'{drive_path}/cod_registro_conselho.csv', sep=';', encoding='ISO-8859-1')
    df_codigo.rename({'cod_conselho_regional': 'codigo_conselho'}, axis=1, inplace=True)
    df_codigo['codigo_conselho'] = df_codigo['codigo_conselho'].astype(str)

    df_profissionais = pd.merge(left=df_profissionais, right=df_codigo, how='left', on='codigo_conselho')
    df_profissionais = df_profissionais[['id_municipio_6', 'conselho_regional', 'qtde_profissional']]
    df_profissionais.rename({'id_municipio_6': 'id_municipio'}, axis=1, inplace=True)

    df_pivot_profissionais = pd.pivot_table(df_profissionais, values='qtde_profissional', index=['id_municipio'],
                            columns=['conselho_regional'], aggfunc='sum', fill_value=0)

    df_pivot_profissionais.columns = [col for col in df_pivot_profissionais.columns]
    df_pivot_profissionais.reset_index(inplace=True)
    df_pivot_profissionais['id_municipio'] = df_pivot_profissionais['id_municipio'].astype(str)

    return df_pivot_profissionais


def gera_df_final(drive_path):

    df_abt_basica = monta_abt(drive_path)

    df_pivot_pop = gera_df_pivot_pop(drive_path)
    df_pib = gera_df_pib(drive_path)
    df_pivot_estabelecimento = pivot_estabelecimento_por_tipo(drive_path)
    df_pivot_profissionais = pivot_profissionais_por_tipo(drive_path)


    df_abt = pd.merge(left=df_abt_basica, right=df_pivot_pop, how='left', on='id_municipio')
    df_abt = pd.merge(left=df_abt, right=df_pivot_estabelecimento, how='left', on='id_municipio')
    df_abt = pd.merge(left=df_abt, right=df_pivot_profissionais, how='left', on='id_municipio')


    df = df_abt[df_abt['qtde_beneficiarios'].notna()]

    df['qtde_beneficiarios'] = df['qtde_beneficiarios'].astype(int)
    df['qtde_operadoras'] = df['qtde_operadoras'].astype(int)
    df['proporcao_beneficiarios'] = df['proporcao_beneficiarios'].apply(lambda x: np.round(x, 2))
    df['cidadao_por_hospital'] = df['cidadao_por_hospital'].astype(int)
    df['beneficiario_por_hospital'] = df['beneficiario_por_hospital'].astype(int)


    df['proporcao_idosos'] = (df['60-64 anos']+df['65-69 anos']+df['70-74 anos']+df['75-79 anos']+df['80-mais'])/df['pop_total']*100
    df['proporcao_idosos'] = df['proporcao_idosos'].apply(lambda x: np.round(x,2))

    df['proporcao_ps'] = (df['Pronto Atendimento']+df['Pronto Socorro Especializado']+df['Pronto Socorro Geral'])/df['qtde_estabelecimentos']*100
    df['proporcao_ps'] = df['proporcao_ps'].apply(lambda x: np.round(x,2))

    df['proporcao_amb'] = df['Clínica Especializada/Ambulatório Especializado']/df['qtde_estabelecimentos']*100
    df['proporcao_amb'] = df['proporcao_amb'].apply(lambda x: np.round(x,2))

    df['qtde_hospitais'] = df['Hospital Especializado']+df['Hospital Geral']+df['Hospital/Dia Isolado']
    df['proporcao_hospital'] = df['qtde_hospitais']/df['qtde_estabelecimentos']*100
    df['proporcao_hospital'] = df['proporcao_hospital'].apply(lambda x: np.round(x,2))

    df['proporcao_apoio'] = (df['Policlínica']+
                            df['Clínica Especializada/Ambulatório Especializado']+df['Unidade de Serviços de Apoio de Diagnose e Terapia']+
                            df['Unidade Móvel Terrestre']+df['Unidade Móvel de Nível Pré-Hospitalar na área de Urgência']+
                            df['Laboratório Central de Saúde Pública - LACEN']+df['Centro de Atenção Hemoterápica e/ou Hematológica']+
                            df['Polo Academia da Saúde']+df['Telessaúde']+df['Central de Regulação de Serviços de Saúde'])/df['pop_total']*100000
    df['proporcao_apoio'] = df['proporcao_apoio'].apply(lambda x: np.round(x,2))

    proporcao_apoio_bh = df.query("nome_municipio == 'Belo Horizonte' ")['proporcao_apoio'].item()
    df['comparacao_bh_apoio'] = np.round((df['proporcao_apoio']/proporcao_apoio_bh)*100, 2)

    df['qtde_profissionais'] = df[' Conselho Regional de Medicina']+df[' Conselho Regional de Psicologia']+df[' Conselho Regional de Enfermagem']+df[' Conselho Regional de Fisioterapia e Terapia Ocupacional']


    df['proporcao_profissionais'] = df['qtde_profissionais']/df['pop_total']*1000
    df['proporcao_profissionais'] = df['proporcao_profissionais'].apply(lambda x: np.round(x,1))

    return df


def import_abt_salva(drive_path):

    df = pd.read_csv(f'{drive_path}/datasets/abt_final/dataset_abt_completa.csv')

    return df

## **Initial setup:**

In [ ]:
# connecting to my Google Drive:
drive.mount('/gdrive')

drive_path = '/gdrive/.shortcut-targets-by-id/1MNI7W_hrAxnm_EKmmeJI9LKcTUIegu2U/ASN.ROCKS'

Mounted at /gdrive


# **2 - [SKIP IF ABT ALREADY SAVED ON DISK] Processing:**

In [ ]:
df = gera_df_final(drive_path)

In [ ]:
df.shape

(3206, 113)

In [ ]:
df.head()

,id_municipio,uf,nome_municipio,regiao_intermediaria,regiao_imediata,mesorregiao,microrregiao,pop_total,qtde_beneficiarios,qtde_operadoras,...,Conselho Regional de Biomedicina,proporcao_idosos,proporcao_ps,proporcao_amb,qtde_hospitais,proporcao_hospital,proporcao_apoio,comparacao_bh_apoio,qtde_profissionais,proporcao_profissionais
151,150010,Pará,Abaetetuba,Belém,Abaetetuba,Nordeste Paraense,Cametá,160439,13524,84,...,11,9.50,0.96,12.50,4,3.85,18.08,17.26,774,4.8
152,150013,Pará,Abel Figueiredo,Marabá,Marabá,Sudeste Paraense,Paragominas,7536,104,21,...,0,10.59,0.00,0.00,1,14.29,0.00,0.00,30,4.0
153,150020,Pará,Acará,Belém,Belém,Nordeste Paraense,Tomé-Açu,55744,470,23,...,2,8.02,0.00,3.23,1,3.23,7.18,6.85,100,1.8
154,150030,Pará,Afuá,Breves,Breves,Marajó,Furos de Breves,39910,135,16,...,2,6.61,0.00,0.00,0,0.00,0.00,0.00,61,1.5
155,150034,Pará,Água Azul do Norte,Redenção,Xinguara,Sudeste Paraense,Parauapebas,27797,450,22,...,2,5.45,0.00,4.55,1,4.55,14.39,13.74,77,2.8


In [ ]:
df.to_csv('dataset_abt_completa.csv', index=False)

# **3 - Presentation:**

In [ ]:
df = import_abt_salva(drive_path)
df.head()

,id_municipio,uf,nome_municipio,regiao_intermediaria,regiao_imediata,mesorregiao,microrregiao,pop_total,qtde_beneficiarios,qtde_operadoras,...,Conselho Regional de Biomedicina,proporcao_idosos,proporcao_ps,proporcao_amb,qtde_hospitais,proporcao_hospital,proporcao_apoio,comparacao_bh_apoio,qtde_profissionais,proporcao_profissionais
0,150010,Pará,Abaetetuba,Belém,Abaetetuba,Nordeste Paraense,Cametá,160439,13524,84,...,11,9.50,0.96,12.50,4,3.85,18.08,17.26,774,4.8
1,150013,Pará,Abel Figueiredo,Marabá,Marabá,Sudeste Paraense,Paragominas,7536,104,21,...,0,10.59,0.00,0.00,1,14.29,0.00,0.00,30,4.0
2,150020,Pará,Acará,Belém,Belém,Nordeste Paraense,Tomé-Açu,55744,470,23,...,2,8.02,0.00,3.23,1,3.23,7.18,6.85,100,1.8
3,150030,Pará,Afuá,Breves,Breves,Marajó,Furos de Breves,39910,135,16,...,2,6.61,0.00,0.00,0,0.00,0.00,0.00,61,1.5
4,150034,Pará,Água Azul do Norte,Redenção,Xinguara,Sudeste Paraense,Parauapebas,27797,450,22,...,2,5.45,0.00,4.55,1,4.55,14.39,13.74,77,2.8


In [ ]:
# GRAFICO COM TODOS OS MUNICIPIOS DE MINAS GERAIS:
df_temp = df.query("uf in ('Minas Gerais') and nome_municipio != 'Extrema' ")
df_temp['symbol'] = np.where(df_temp['nome_municipio'] == 'Belo Horizonte', '0', '1')

fig = px.scatter(
    df_temp, x='comparacao_bh_apoio', y='proporcao_beneficiarios', opacity=0.8,
    size='pib_per_capita', color='regiao_intermediaria', hover_name='nome_municipio',
    #symbol='symbol',
    #symbol_sequence= ['circle', 'star'],
    #title='Situação dos Municípios em 2021',
    log_x=False , color_discrete_sequence=px.colors.qualitative.Dark24,
    #range_x=[0, 4],
    range_y=[-5,100],
    size_max=30,
    labels=dict(
        proporcao_beneficiarios='Beneficiários (%)',
        uf='Estado',
        regiao_intermediaria='Região Intermediária',
        mesorregiao='Mesorregião',
        pop_total='Nº de habitantes',
        qtde_operadoras='Quantidade de operadoras',
        beneficiario_por_hospital='Nº beneficiários a cada 1 hospital',
        cidadao_por_hospital='Nº cidadãos a cada 1 hospital',
        proporcao_profissionais='Oferta de Profissionais de Saúde para cada 1000 pessoas',
        proporcao_apoio='Rede de Apoio (nº de estabelecimentos de apoio para cada 1 hospital)',
        pib_per_capita='PIB per capita (R$)',
        comparacao_bh_apoio='Tamanho da rede de apoio, em relação à BH (em %)'
    )
)

# ***********************************************************************************************
# ***********************************************************************************************
# TODO:
# - PESQUISAR COMO PINTAR UMA BOLHA ESPECIFICA DE UMA COR DIFERENTE OU DAR OUTRO DESTAQUE!!!
# - AUMENTAR FONTE DOS EIXOS E DA LEGENDA (COLOCAR EM NEGRITO TAMBEM)!!!
# - FORMATAR MELHOR O TOOLTIP AO PASSAR O MOUSE EM CADA BOLHA!!!
# ***********************************************************************************************
# ***********************************************************************************************


#fig.update_layout({
#    'plot_bgcolor': 'rgb(255, 255, 255)',
#    'paper_bgcolor': 'rgb(255, 255, 255)',
#})

fig.show()

In [ ]:
# GRAFICO COMPARANDO OUTROS ESTADOS COM MINAS GERAIS (PRINCIPALMENTE BELO HORIZONTE):
df_outros_estados = df.query("uf in ('Minas Gerais', 'Espírito Santo', 'Bahia', 'Goiás') ")
df_outros_estados['symbol'] = np.where(df_outros_estados['nome_municipio'] == 'Belo Horizonte', '0', '1')

fig = px.scatter(
    df_outros_estados, x='comparacao_bh_apoio', y='proporcao_beneficiarios', opacity=0.8,
    size='pib_per_capita', color='uf', hover_name='nome_municipio',
    #symbol='symbol',
    #symbol_sequence= ['circle', 'star'],
    #title='Situação dos Municípios em 2021',
    log_x=False , color_discrete_sequence=px.colors.qualitative.Dark24,
    #range_x=[0, 4],
    range_y=[-5,100],
    size_max=30,
    labels=dict(
        proporcao_beneficiarios='Beneficiários (%)',
        uf='Estado',
        regiao_intermediaria='Região Intermediária',
        mesorregiao='Mesorregião',
        pop_total='Nº de habitantes',
        qtde_operadoras='Quantidade de operadoras',
        beneficiario_por_hospital='Nº beneficiários a cada 1 hospital',
        cidadao_por_hospital='Nº cidadãos a cada 1 hospital',
        proporcao_profissionais='Oferta de Profissionais de Saúde para cada 1000 pessoas',
        proporcao_apoio='Rede de Apoio (nº de estabelecimentos de apoio para cada 1000 pessoas)',
        pib_per_capita='PIB per capita (R$)',
        comparacao_bh_apoio='Tamanho da rede de apoio, em relação à BH (em %)'
    )
)

fig.update_layout({
    #'plot_bgcolor': 'rgb(255, 255, 255)'
    #'paper_bgcolor': 'rgb(255, 255, 255)'
})

fig.show()

In [ ]:
df_outros_estados.query("nome_municipio == 'Montes Claros'")[['pop_total', 'valor_pib', 'pib_per_capita', 'proporcao_beneficiarios']]

,pop_total,valor_pib,pib_per_capita,proporcao_beneficiarios
1909,417478,9714238.0,23731.41,28.48


In [ ]:
%%shell
jupyter nbconvert --to slides /content/Datathon_ASN_Montando_ABT_turbinada.ipynb